# Team Members

* **Student ID:** 0001111416  
  **Full Name:** Alessio Pittiglio  
  **Institutional Email:** alessio.pittiglio@studio.unibo.it

* **Student ID:** 0001086355  
  **Full Name:** Parsa Mastouri Kashani  
  **Institutional Email:** parsa.mastouri@studio.unibo.it


# **Product Recognition of Books**

## Image Processing and Computer Vision - Assignment Module \#1


Contacts:

- Prof. Giuseppe Lisanti -> giuseppe.lisanti@unibo.it
- Prof. Samuele Salti -> samuele.salti@unibo.it
- Alex Costanzino -> alex.costanzino@unibo.it
- Francesco Ballerini -> francesco.ballerini4@unibo.it

Computer vision-based object detection techniques can be applied in library or bookstore settings to build a system that identifies books on shelves.

Such a system could assist in:
* Helping visually impaired users locate books by title/author;
* Automating inventory management (e.g., detecting misplaced or out-of-stock books);
* Enabling faster book retrieval by recognizing spine text or cover designs.

## Task
Develop a computer vision system that, given a reference image for each book, is able to identify such book from one picture of a shelf.

<figure>
<a href="https://ibb.co/pvLVjbM5"><img src="https://i.ibb.co/svVx9bNz/example.png" alt="example" border="0"></a>
</figure>

For each type of product displayed on the shelf, the system should compute a bounding box aligned with the book spine or cover and report:
1. Number of instances;
1. Dimension of each instance (area in pixel of the bounding box that encloses each one of them);
1. Position in the image reference system of each instance (four corners of the bounding box that enclose them);
1. Overlay of the bounding boxes on the scene images.

<font color="red"><b>Each step of this assignment must be solved using traditional computer vision techniques.</b></font>

#### Example of expected output
```
Book 0 - 2 instance(s) found:
  Instance 1 {top_left: (100,200), top_right: (110, 220), bottom_left: (10, 202), bottom_right: (10, 208), area: 230px}
  Instance 2 {top_left: (90,310), top_right: (95, 340), bottom_left: (24, 205), bottom_right: (23, 234), area: 205px}
Book 1 – 1 instance(s) found:
.
.
.
```

## Data
Two folders of images are provided:
* **Models**: contains one reference image for each product that the system should be able to identify;
* **Scenes**: contains different shelve pictures to test the developed algorithm in different scenarios.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !cp -r /content/drive/MyDrive/AssignmentsIPCV/dataset.zip ./
# !unzip dataset.zip

## Evaluation criteria
1. **Clarity and conciseness**. Present your work in a readable way: format your code and comment every important step;

2. **Procedural correctness**. There are several ways to solve the assignment. Design your own sound approach and justify every decision you make;

3. **Correctness of results**. Try to solve as many instances as possible. You should be able to solve all the instances of the assignment, however, a thoroughly justified and sound procedure with a lower number of solved instances will be valued **more** than a poorly designed and justified approach that solves more or all instances.

---

## Solution

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from ght_sift_lib import SIFT_GHT_Detector, natural_sort_key, format_and_print_results

In [ ]:
MODELS_DIR = os.path.join("dataset", "models")
SCENES_DIR = os.path.join("dataset", "scenes")

In [ ]:
detector = SIFT_GHT_Detector()

In [ ]:
models = {}
model_files = sorted(
    [
        f
        for f in os.listdir(MODELS_DIR)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ],
    key=natural_sort_key,
)

In [ ]:
for i, filename in enumerate(model_files):
    book_name = f"Book {i}"
    models[book_name] = cv2.imread(os.path.join(MODELS_DIR, filename))

In [ ]:
scene_files = sorted(os.listdir(SCENES_DIR), key=natural_sort_key)

In [ ]:
for scene_filename in scene_files:
    if not scene_filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    scene_path = os.path.join(SCENES_DIR, scene_filename)
    scene_image = cv2.imread(scene_path)
    overlay_image = scene_image.copy()

    all_scene_detections = {}
    for book_name, model_image in models.items():
        detections = detector.detect_multiple(model_image, scene_image, verbose=False)

        if detections:
            all_scene_detections[book_name] = detections
            # Draw bounding boxes
            for det in detections:
                cv2.polylines(
                    overlay_image,
                    [det["corners_list"]],
                    isClosed=True,
                    color=(0, 255, 0),
                    thickness=2,
                )

    format_and_print_results(all_scene_detections, scene_filename)

    # Display results
    plt.imshow(cv2.cvtColor(overlay_image, cv2.COLOR_BGR2RGB))
    plt.show()

# Example: Detecting instances of a single book

In [ ]:
# Example scene and model
scene_filename = scene_files[3]
scene_path = os.path.join(SCENES_DIR, scene_filename)
scene_image = cv2.imread(scene_path)
model_image = models["Book 16"]

# Detect and visualize
detections = detector.detect_multiple(model_image, scene_image, verbose=False)
overlay_image = scene_image.copy()

for det in detections:
    cv2.polylines(
        overlay_image,
        [det["corners_list"]],
        isClosed=True,
        color=(0, 255, 0),
        thickness=2,
    )

plt.imshow(cv2.cvtColor(overlay_image, cv2.COLOR_BGR2RGB))
plt.title(f"Detections in {scene_filename}")
plt.show()